# Ajustar las Bases de Datos de Indicadores Territoriales

In [4]:
# Cargar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import TerriDataUtility as tdu

In [2]:
# Cargar codigos divipola
departamentos = pd.read_csv("Datos/cod_divipola_departamento.csv", sep=";")
departamentos.head()

,Código,Nombre
0,5,ANTIOQUIA
1,8,ATLÁNTICO
2,11,"BOGOTÁ, D.C."
3,13,BOLÍVAR
4,15,BOYACÁ


In [3]:
# Cargar datos de municipios
municipios = pd.read_csv("Datos/cod_divipola_municipio.csv", sep=";")
municipios.head()

,Codigo Departamento,Departamento,Codigo Municipio,Municipio,Tipo
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio


## Funciones

### Rango de año

In [4]:
# Revisar para cada subcategoria el rango de años en los que se tienen datos disponibles
def year_range_ind(data,subcategoria,departamental=True):
    
    if(departamental):
        codEntidades = departamentos['Código']*1000
    else:
        codEntidades = municipios['Codigo Municipio']
        
    # Filtrar la base de datos para acceso
    dataSubcategoria = data.loc[(data['Subcategoría'] == subcategoria) & (data['Código Entidad'].isin(codEntidades))]
    
    # Verificar los años en los que se tienen registrado datos para los indicadores
    yearMinMax = dataSubcategoria.groupby('Indicador').agg({'Año':[np.min, np.max]}).reset_index()
    yearMinMax.columns = ["_".join(pair) for pair in yearMinMax.columns]
    return(yearMinMax)

### Subcategorias e indicadores

In [5]:
# Extraer subcategorias de la dimensión
def subcat_dim(data,departamental=True):
    if(departamental):
        codEntidades = departamentos['Código']*1000
    else:
        codEntidades = municipios['Codigo Municipio']
    
    # Filtrar la base de datos para acceso
    dataSub = data.loc[data['Código Entidad'].isin(codEntidades)]
    
    subcatDim = list(pd.unique(dataSub['Subcategoría']))
    return(subcatDim)

In [6]:
# Extraer indicadores de subcategorias
def ind_subcat(data,subcategoria,departamental=True):
    
    if(departamental):
        codEntidades = departamentos['Código']*1000
    else:
        codEntidades = municipios['Codigo Municipio']
        
    # Filtrar la base de datos para acceso
    dataSubcategoria = data.loc[(data['Subcategoría'] == subcategoria) & (data['Código Entidad'].isin(codEntidades))]

    indSubcat = list(dataSubcategoria.loc[educacion['Subcategoría'] == subcategoria]['Indicador'].unique())
    return(indSubcat)

In [7]:
# Imprimir las subcategorias e indicadores de una dimensión
def print_subcat_ind(data,departamental=True):
    subCat = subcat_dim(data,departamental=departamental)
    for i,j in zip(subCat, range(len(subCat))):
        print(f"{j+1} {i}")
        subCatInd = ind_subcat(data=data,subcategoria=i,departamental=departamental)
        for k,l in zip(subCatInd, range(len(subCatInd))):
            print(f"   {j+1}.{l+1} {k}")

### Agregar Indicadores

In [8]:
# Calcular indicadores agregados segun la subcategoria, el nivel y el año
def agg_ind(data,subcategoria,yearMin,yearMax,departamental=True):
    
    if(departamental):
        codEntidades = departamentos['Código']*1000
    else:
        codEntidades = municipios['Codigo Municipio']
    
    # Filtrar la base de datos segun la subcategoria, el codigo de la entidad y el año
    dataSubcategoria = data.loc[(data['Subcategoría'] == subcategoria) & 
                                (data['Código Entidad'].isin(codEntidades)) &
                                (data['Año'] >= yearMin) &
                                (data['Año'] <= yearMax)]
    
    # Extraer los indicadores de la subcategoria
    indSub = ind_subcat(data,subcategoria,departamental)
    
#    if(departamental):
#        meanAcceso = pd.DataFrame({'Entidad':dataSubcategoria['Entidad'].unique()})
#        for i in indSub:
#            temp = dataSubcategoria.loc[dataSubcategoria['Indicador'] == i]\
#            .groupby(['Código Entidad','Entidad'])\
#            .agg({'Dato Numérico': 'mean'})\
#            .rename(columns={'Dato Numérico':i})\
#            .reset_index()
#            meanAcceso = pd.merge(left=meanAcceso, right=temp, on='Entidad', how='left')
#    else:
    meanAcceso = dataSubcategoria.groupby(['Código Entidad','Entidad']).size().reset_index().rename(columns = {0:'Numero de Registros'})
    meanAcceso.drop(columns = ['Numero de Registros'], inplace=True)
    for i in indSub:
        temp = dataSubcategoria.loc[dataSubcategoria['Indicador'] == i].groupby(['Código Entidad','Entidad']).agg({'Dato Numérico': 'mean'}).rename(columns={'Dato Numérico':i}).reset_index()
        meanAcceso = pd.merge(left=meanAcceso, right=temp, on=['Código Entidad','Entidad'], how='left')
    
    return(meanAcceso)

### Gráficas

In [9]:
# Gráficar el comportamiento del indicador en el tiempo
def plot_line(data,indicadores,codEntidad):
    data_plot = data.loc[(data['Código Entidad'] == codEntidad) & (data['Indicador'].isin(indicadores))]
    fig = px.line(data_plot, x="Año", y="Dato Numérico", color='Indicador')
    fig.show()

## Estrategia de Análisis

Con respecto a estos indicadores se propone lo siguiente:
- Hacer un análisis por nivel de la entidad: Nacional, Departamental y Municipal
- Construir un indicador con el promedio por nivel con los años registrados
- Construir un indicador que indique la tendencia del indicador con los años registrados

# 1. Educación

In [10]:
# Cargar datos de educacion
colType = {
    'Código Departamento':'int64',
    'Departamento':'object',
    'Código Entidad':'int64',
    'Entidad':'object',
    'Dimensión':'object',
    'Subcategoría':'object',
    'Dato Numérico':'float64',
    'Dato Cualitativo':'float64',
    'Año':'int64',
    'Mes':'int64',
    'Fuente':'object',
    'Unidad de Medida':'object'}   

educacion = pd.read_csv('Datos/TerriData_Educacion.gz', 
                        compression='gzip',
                        header=0, 
                        sep=';', 
                        quotechar='"',
                        dtype = colType)
educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255138 entries, 0 to 255137
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Código Departamento  255138 non-null  int64  
 1   Departamento         255138 non-null  object 
 2   Código Entidad       255138 non-null  int64  
 3   Entidad              255138 non-null  object 
 4   Dimensión            255138 non-null  object 
 5   Subcategoría         255138 non-null  object 
 6   Indicador            255138 non-null  object 
 7   Dato Numérico        233108 non-null  float64
 8   Dato Cualitativo     0 non-null       float64
 9   Año                  255138 non-null  int64  
 10  Mes                  255138 non-null  int64  
 11  Fuente               255138 non-null  object 
 12  Unidad de Medida     255138 non-null  object 
dtypes: float64(2), int64(4), object(7)
memory usage: 25.3+ MB


In [11]:
educacion.head()

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Educación,Acceso a la educación,Cobertura bruta en transición,95.42,NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
1,5,Antioquia,5000,Antioquia,Educación,Acceso a la educación,Cobertura bruta en transición,78.62,NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
2,5,Antioquia,5001,Medellín,Educación,Acceso a la educación,Cobertura bruta en transición,103.23,NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
3,5,Antioquia,5002,Abejorral,Educación,Acceso a la educación,Cobertura bruta en transición,51.80,NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
4,5,Antioquia,5004,Abriaquí,Educación,Acceso a la educación,Cobertura bruta en transición,96.00,NaN,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)


### Subcategorias e Indicadores a Nivel departamental

In [12]:
print_subcat_ind(data=educacion,departamental=True)

1 Acceso a la educación
   1.1 Cobertura bruta en transición
   1.2 Cobertura bruta en educación primaria
   1.3 Cobertura bruta en educación secundaria
   1.4 Cobertura bruta en educación media
   1.5 Cobertura bruta en educación básica
   1.6 Cobertura bruta en educación - Total
   1.7 Cobertura neta en transición
   1.8 Cobertura neta en educación primaria
   1.9 Cobertura neta en educación secundaria
   1.10 Cobertura neta en educación media
   1.11 Cobertura neta en educación básica
   1.12 Cobertura neta en educación - Total
   1.13 Cobertura en educación superior
2 Permanencia y rezago
   2.1 Tasa de deserción intra-anual del sector oficial en educación básica y media (Desde transición hasta once)
   2.2 Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)
3 Logro
   3.1 Años de educación personas de 15 y más años
   3.2 Tasa de analfabetismo de personas de 15 y más años
   3.3 Tasa de Analfabetismo (Censo)
   3.4 Tasa de Analfabetismo 

#### Acceso a la educación

In [13]:
year_range_ind(data=educacion, subcategoria='Acceso a la educación',departamental=True)

,Indicador_,Año_amin,Año_amax
0,Cobertura bruta en educación - Total,2005,2018
1,Cobertura bruta en educación básica,2005,2018
2,Cobertura bruta en educación media,2005,2018
3,Cobertura bruta en educación primaria,2005,2018
4,Cobertura bruta en educación secundaria,2005,2018
5,Cobertura bruta en transición,2005,2018
6,Cobertura en educación superior,2005,2018
7,Cobertura neta en educación - Total,2005,2018
8,Cobertura neta en educación básica,2005,2018
9,Cobertura neta en educación media,2005,2018


In [14]:
accesoDep = agg_ind(data=educacion,subcategoria='Acceso a la educación',yearMin=2008,yearMax=2018,departamental=True)
accesoDep

,Código Entidad,Entidad,Cobertura bruta en transición,Cobertura bruta en educación primaria,Cobertura bruta en educación secundaria,Cobertura bruta en educación media,Cobertura bruta en educación básica,Cobertura bruta en educación - Total,Cobertura neta en transición,Cobertura neta en educación primaria,Cobertura neta en educación secundaria,Cobertura neta en educación media,Cobertura neta en educación básica,Cobertura neta en educación - Total,Cobertura en educación superior
0,5000,Antioquia,83.603636,113.086364,108.840909,81.600909,109.162727,103.770909,65.508182,90.254545,74.313636,42.535455,91.647273,90.375455,47.847273
1,8000,Atlántico,89.180000,109.362727,99.878182,80.690000,104.639091,99.775455,54.538182,88.318182,74.929091,46.712727,91.640000,90.717273,48.710909
2,13000,Bolívar,103.279091,118.990909,103.351818,77.590000,112.654545,105.596364,60.850909,88.903636,69.600909,39.496364,91.809091,90.618182,33.036364
3,15000,Boyacá,83.339091,100.136364,105.800000,87.899091,101.368182,98.745455,63.288182,81.697273,74.511818,48.407273,84.671818,84.801818,47.221818
4,17000,Caldas,75.348182,96.732727,99.802727,80.308182,96.130000,93.179091,58.857273,78.208182,70.627273,43.558182,81.874545,82.014545,44.372727
5,18000,Caquetá,82.345455,114.262727,87.915455,60.534545,103.332727,94.289091,50.475455,81.071818,54.647273,24.191818,82.294545,80.005455,22.041818
6,19000,Cauca,82.686364,120.303636,99.692727,70.541818,110.909091,101.824545,49.546364,86.295455,62.472727,30.904545,86.770000,84.150909,28.722727
7,20000,Cesar,113.987273,129.250909,103.418182,75.071818,120.256364,110.543636,62.147273,93.478182,69.693636,36.950909,97.010909,94.834545,28.111818
8,23000,Córdoba,101.250000,122.486364,107.343636,77.641818,116.270909,108.270909,59.518182,90.433636,70.720000,37.065455,93.215455,92.273636,20.689091
9,25000,Cundinamarca,86.840909,104.030909,109.142727,83.908182,104.358182,100.930909,62.120000,88.174545,80.464545,47.785455,91.531818,90.915455,25.770000


#### Permanencia y rezago

In [15]:
year_range_ind(data=educacion, subcategoria='Permanencia y rezago',departamental=True)

,Indicador_,Año_amin,Año_amax
0,Tasa de deserción intra-anual del sector ofici...,2003,2018
1,Tasa de repitencia del sector oficial en educa...,2003,2018


In [16]:
permanenciaDep = agg_ind(data=educacion,subcategoria='Permanencia y rezago',yearMin=2008,yearMax=2018,departamental=True)
permanenciaDep

,Código Entidad,Entidad,Tasa de deserción intra-anual del sector oficial en educación básica y media (Desde transición hasta once),Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)
0,5000,Antioquia,4.027273,2.848182
1,8000,Atlántico,2.731818,1.576364
2,13000,Bolívar,3.656364,2.794545
3,15000,Boyacá,2.835455,2.365455
4,17000,Caldas,4.058182,3.125455
5,18000,Caquetá,6.512727,2.486364
6,19000,Cauca,3.583636,1.961818
7,20000,Cesar,5.091818,2.413636
8,23000,Córdoba,3.635455,2.825455
9,25000,Cundinamarca,3.504545,2.380000


#### Logro

In [17]:
year_range_ind(data=educacion, subcategoria='Logro', departamental=True)

,Indicador_,Año_amin,Año_amax
0,Años de educación personas de 15 y más años,2014,2018
1,Porcentaje de asistencia de 5 a 24 años (Censo),2018,2018
2,Porcentaje de asistencia de 5 a 24 años Rural ...,2018,2018
3,Porcentaje de asistencia de 5 a 24 años Urbana...,2018,2018
4,Tasa de Analfabetismo (Censo),2018,2018
5,Tasa de Analfabetismo Rural (Censo),2018,2018
6,Tasa de Analfabetismo Urbana (Censo),2018,2018
7,Tasa de analfabetismo de personas de 15 y más ...,2014,2018


In [18]:
logroDep = agg_ind(data=educacion, subcategoria='Logro', yearMin=2014, yearMax=2018, departamental=True)
logroDep

,Código Entidad,Entidad,Años de educación personas de 15 y más años,Tasa de analfabetismo de personas de 15 y más años,Tasa de Analfabetismo (Censo),Tasa de Analfabetismo Urbana (Censo),Tasa de Analfabetismo Rural (Censo),Porcentaje de asistencia de 5 a 24 años (Censo),Porcentaje de asistencia de 5 a 24 años Urbana (Censo),Porcentaje de asistencia de 5 a 24 años Rural (Censo)
0,5000,Antioquia,8.878,4.868,4.46,3.00,10.79,66.89,67.97,63.29
1,8000,Atlántico,9.724,3.952,3.65,3.17,13.38,68.31,68.46,65.58
2,13000,Bolívar,8.624,8.444,7.77,5.49,15.29,67.72,68.04,66.85
3,15000,Boyacá,8.086,5.558,5.84,2.98,10.17,73.00,76.79,67.12
4,17000,Caldas,8.232,5.184,5.07,3.72,9.42,68.90,70.93,63.41
5,18000,Caquetá,7.396,6.648,6.89,6.02,8.92,65.81,68.75,59.42
6,19000,Cauca,7.300,7.832,7.53,3.87,10.10,66.37,72.10,63.16
7,20000,Cesar,8.262,9.838,8.69,6.41,16.46,68.67,70.08,64.58
8,23000,Córdoba,7.698,12.562,11.55,7.09,17.53,70.75,72.52,68.69
9,25000,Cundinamarca,8.596,3.486,3.35,2.15,6.61,70.01,70.39,68.93


#### Calidad

In [19]:
year_range_ind(data=educacion, subcategoria='Calidad', departamental=True)

,Indicador_,Año_amin,Año_amax
0,Puntaje promedio Pruebas Saber 11 - Lectura cr...,2010,2018
1,Puntaje promedio Pruebas Saber 11 - Matemáticas,2010,2018


In [20]:
calidadDep = agg_ind(data=educacion, subcategoria='Calidad', yearMin=2010, yearMax=2018, departamental=True)
calidadDep

,Código Entidad,Entidad,Puntaje promedio Pruebas Saber 11 - Matemáticas,Puntaje promedio Pruebas Saber 11 - Lectura crítica
0,5000,Antioquia,47.677778,49.708889
1,8000,Atlántico,47.667778,48.766667
2,13000,Bolívar,45.447778,46.856667
3,15000,Boyacá,51.270000,50.692222
4,17000,Caldas,48.624444,50.087778
5,18000,Caquetá,46.196667,47.311111
6,19000,Cauca,45.523333,47.020000
7,20000,Cesar,46.581111,47.656667
8,23000,Córdoba,46.231111,47.445556
9,25000,Cundinamarca,49.325556,50.183333


In [21]:
bases = [accesoDep, logroDep, permanenciaDep, calidadDep]
for i in bases:
    i.set_index(['Código Entidad','Entidad'], inplace=True)
educacionDep = pd.concat(bases, axis=1, join='outer')
educacionDep.reset_index(inplace=True)
#educacionDep.rename(columns={'index':'Entidad'}, inplace=True)

In [22]:
educacionDep.to_csv("Datos/educacionDep.csv", index=False)

### Subcategorias e Indicadores a Nivel Municipal

Los indicadores que se miden a nivel departamental son:

In [23]:
print_subcat_ind(data=educacion,departamental=False)

1 Acceso a la educación
   1.1 Cobertura bruta en transición
   1.2 Cobertura bruta en educación primaria
   1.3 Cobertura bruta en educación secundaria
   1.4 Cobertura bruta en educación media
   1.5 Cobertura bruta en educación básica
   1.6 Cobertura bruta en educación - Total
   1.7 Cobertura neta en transición
   1.8 Cobertura neta en educación primaria
   1.9 Cobertura neta en educación secundaria
   1.10 Cobertura neta en educación media
   1.11 Cobertura neta en educación básica
   1.12 Cobertura neta en educación - Total
2 Permanencia y rezago
   2.1 Tasa de deserción intra-anual del sector oficial en educación básica y media (Desde transición hasta once)
   2.2 Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)
3 Logro
   3.1 Tasa de Analfabetismo (Censo)
   3.2 Tasa de Analfabetismo Urbana (Censo)
   3.3 Tasa de Analfabetismo Rural (Censo)
   3.4 Porcentaje de asistencia de 5 a 24 años (Censo)
   3.5 Porcentaje de asistencia de 5

#### Acceso a la educación

In [24]:
year_range_ind(data=educacion, subcategoria='Acceso a la educación',departamental=False)

,Indicador_,Año_amin,Año_amax
0,Cobertura bruta en educación - Total,2005,2018
1,Cobertura bruta en educación básica,2005,2018
2,Cobertura bruta en educación media,2005,2018
3,Cobertura bruta en educación primaria,2005,2018
4,Cobertura bruta en educación secundaria,2005,2018
5,Cobertura bruta en transición,2005,2018
6,Cobertura neta en educación - Total,2005,2018
7,Cobertura neta en educación básica,2005,2018
8,Cobertura neta en educación media,2005,2018
9,Cobertura neta en educación primaria,2005,2018


In [25]:
accesoMun = agg_ind(data=educacion,subcategoria='Acceso a la educación',yearMin=2008,yearMax=2018,departamental=False)
accesoMun

,Código Entidad,Entidad,Cobertura bruta en transición,Cobertura bruta en educación primaria,Cobertura bruta en educación secundaria,Cobertura bruta en educación media,Cobertura bruta en educación básica,Cobertura bruta en educación - Total,Cobertura neta en transición,Cobertura neta en educación primaria,Cobertura neta en educación secundaria,Cobertura neta en educación media,Cobertura neta en educación básica,Cobertura neta en educación - Total
0,5001,Medellín,100.723636,122.357273,133.155455,106.534545,124.771818,121.417273,83.374545,104.451818,92.396364,56.267273,106.371818,105.770000
1,5002,Abejorral,72.562727,123.012727,97.767273,66.504545,107.880909,101.020000,52.411818,96.441818,68.650000,34.159091,92.798182,89.082727
2,5004,Abriaquí,86.954545,113.481818,108.535455,78.008182,108.874545,103.508182,65.740000,92.820909,76.397273,42.588182,93.313636,91.560000
3,5021,Alejandría,110.370909,159.231818,126.330000,83.976364,140.880909,130.281818,93.789091,126.046364,84.040000,41.122727,120.058182,114.030909
4,5030,Amagá,57.044545,77.628182,81.497273,63.466364,85.866364,74.647273,49.323636,71.701818,64.497273,37.755455,74.521818,74.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,97666,Taraira,68.916000,174.498000,138.340000,76.659000,148.594000,138.273000,31.174545,103.637273,60.987273,16.204545,110.395455,110.606364
1097,99001,Puerto Carreño,92.902000,124.605000,100.619000,73.667000,111.921000,105.513000,57.219091,91.937273,64.590909,30.954545,93.633636,91.687273
1098,99524,La Primavera,40.806000,77.999000,56.428000,35.846000,65.810000,61.190000,25.066364,57.249091,38.327273,18.186364,57.940909,56.695455
1099,99624,Santa Rosalía,51.920000,106.613000,70.807000,38.614000,87.346000,80.210000,27.964545,73.123636,43.060909,17.680000,73.570909,72.243636


#### Permanencia y rezago

In [26]:
year_range_ind(data=educacion, subcategoria='Permanencia y rezago',departamental=False)

,Indicador_,Año_amin,Año_amax
0,Tasa de deserción intra-anual del sector ofici...,2003,2018
1,Tasa de repitencia del sector oficial en educa...,2003,2018


In [27]:
permanenciaMun = agg_ind(data=educacion,subcategoria='Permanencia y rezago',yearMin=2008,yearMax=2018,departamental=False)
permanenciaMun

,Código Entidad,Entidad,Tasa de deserción intra-anual del sector oficial en educación básica y media (Desde transición hasta once),Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)
0,5001,Medellín,3.42000,3.74375
1,5002,Abejorral,3.88000,3.05125
2,5004,Abriaquí,2.38625,1.89625
3,5021,Alejandría,2.41750,2.20375
4,5030,Amagá,4.48500,3.25250
...,...,...,...,...
1096,97666,Taraira,8.68250,3.71500
1097,99001,Puerto Carreño,6.95375,3.66875
1098,99524,La Primavera,5.98875,3.55875
1099,99624,Santa Rosalía,6.22375,2.42375


#### Logro

In [28]:
year_range_ind(data=educacion, subcategoria='Logro',departamental=False)

,Indicador_,Año_amin,Año_amax
0,Porcentaje de asistencia de 5 a 24 años (Censo),2018,2018
1,Porcentaje de asistencia de 5 a 24 años Rural ...,2018,2018
2,Porcentaje de asistencia de 5 a 24 años Urbana...,2018,2018
3,Tasa de Analfabetismo (Censo),2018,2018
4,Tasa de Analfabetismo Rural (Censo),2018,2018
5,Tasa de Analfabetismo Urbana (Censo),2018,2018


In [29]:
logroMun = agg_ind(data=educacion,subcategoria='Logro',yearMin=2008,yearMax=2018,departamental=False)
logroMun

,Código Entidad,Entidad,Tasa de Analfabetismo (Censo),Tasa de Analfabetismo Urbana (Censo),Tasa de Analfabetismo Rural (Censo),Porcentaje de asistencia de 5 a 24 años (Censo),Porcentaje de asistencia de 5 a 24 años Urbana (Censo),Porcentaje de asistencia de 5 a 24 años Rural (Censo)
0,5002,Abejorral,10.15,8.76,11.10,63.96,68.13,61.54
1,5004,Abriaquí,6.97,4.07,8.41,65.97,69.41,64.32
2,5021,Alejandría,8.63,6.22,12.06,68.63,74.15,61.83
3,5030,Amagá,7.58,5.22,9.84,60.64,59.41,61.74
4,5031,Amalfi,10.41,7.54,14.43,63.69,68.13,57.95
...,...,...,...,...,...,...,...,...
1065,97161,Carurú,10.08,5.07,14.77,66.52,69.57,64.61
1066,97666,Taraira,15.18,2.84,24.43,56.10,71.07,49.67
1067,99524,La Primavera,7.81,5.81,10.54,60.66,71.56,44.43
1068,99624,Santa Rosalía,7.86,7.00,9.30,67.38,71.74,60.23


#### Calidad

In [30]:
year_range_ind(data=educacion, subcategoria='Calidad',departamental=False)

,Indicador_,Año_amin,Año_amax
0,Puntaje promedio Pruebas Saber 11 - Lectura cr...,2010,2018
1,Puntaje promedio Pruebas Saber 11 - Matemáticas,2010,2018


In [31]:
calidadMun = agg_ind(data=educacion,subcategoria='Calidad',yearMin=2008,yearMax=2018,departamental=False)
calidadMun

,Código Entidad,Entidad,Puntaje promedio Pruebas Saber 11 - Matemáticas,Puntaje promedio Pruebas Saber 11 - Lectura crítica
0,5001,Medellín,48.716667,50.837778
1,5002,Abejorral,47.035556,49.226667
2,5004,Abriaquí,43.757778,46.960000
3,5021,Alejandría,45.346667,48.550000
4,5030,Amagá,45.146667,48.104444
...,...,...,...,...
1096,97666,Taraira,42.470000,43.653333
1097,99001,Puerto Carreño,45.168889,46.876667
1098,99524,La Primavera,46.646667,47.755556
1099,99624,Santa Rosalía,46.688889,46.870000


In [32]:
# Integrar bases de datos de indicadores de la Subcategoria educación a nivel municipal
bases = [accesoMun, permanenciaMun, logroMun, calidadMun]
for i in bases:
    i.set_index(['Código Entidad','Entidad'], inplace=True)
educacionMun = pd.concat(bases, axis=1, join='outer')
educacionMun.reset_index(inplace=True)
educacionMun

,Código Entidad,Entidad,Cobertura bruta en transición,Cobertura bruta en educación primaria,Cobertura bruta en educación secundaria,Cobertura bruta en educación media,Cobertura bruta en educación básica,Cobertura bruta en educación - Total,Cobertura neta en transición,Cobertura neta en educación primaria,...,Tasa de deserción intra-anual del sector oficial en educación básica y media (Desde transición hasta once),Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once),Tasa de Analfabetismo (Censo),Tasa de Analfabetismo Urbana (Censo),Tasa de Analfabetismo Rural (Censo),Porcentaje de asistencia de 5 a 24 años (Censo),Porcentaje de asistencia de 5 a 24 años Urbana (Censo),Porcentaje de asistencia de 5 a 24 años Rural (Censo),Puntaje promedio Pruebas Saber 11 - Matemáticas,Puntaje promedio Pruebas Saber 11 - Lectura crítica
0,5001,Medellín,100.723636,122.357273,133.155455,106.534545,124.771818,121.417273,83.374545,104.451818,...,3.42000,3.74375,NaN,NaN,NaN,NaN,NaN,NaN,48.716667,50.837778
1,5002,Abejorral,72.562727,123.012727,97.767273,66.504545,107.880909,101.020000,52.411818,96.441818,...,3.88000,3.05125,10.15,8.76,11.10,63.96,68.13,61.54,47.035556,49.226667
2,5004,Abriaquí,86.954545,113.481818,108.535455,78.008182,108.874545,103.508182,65.740000,92.820909,...,2.38625,1.89625,6.97,4.07,8.41,65.97,69.41,64.32,43.757778,46.960000
3,5021,Alejandría,110.370909,159.231818,126.330000,83.976364,140.880909,130.281818,93.789091,126.046364,...,2.41750,2.20375,8.63,6.22,12.06,68.63,74.15,61.83,45.346667,48.550000
4,5030,Amagá,57.044545,77.628182,81.497273,63.466364,85.866364,74.647273,49.323636,71.701818,...,4.48500,3.25250,7.58,5.22,9.84,60.64,59.41,61.74,45.146667,48.104444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,97666,Taraira,68.916000,174.498000,138.340000,76.659000,148.594000,138.273000,31.174545,103.637273,...,8.68250,3.71500,15.18,2.84,24.43,56.10,71.07,49.67,42.470000,43.653333
1097,99001,Puerto Carreño,92.902000,124.605000,100.619000,73.667000,111.921000,105.513000,57.219091,91.937273,...,6.95375,3.66875,NaN,NaN,NaN,NaN,NaN,NaN,45.168889,46.876667
1098,99524,La Primavera,40.806000,77.999000,56.428000,35.846000,65.810000,61.190000,25.066364,57.249091,...,5.98875,3.55875,7.81,5.81,10.54,60.66,71.56,44.43,46.646667,47.755556
1099,99624,Santa Rosalía,51.920000,106.613000,70.807000,38.614000,87.346000,80.210000,27.964545,73.123636,...,6.22375,2.42375,7.86,7.00,9.30,67.38,71.74,60.23,46.688889,46.870000


In [33]:
# Copiar la base de datos a formato csv
educacionMun.to_csv("Datos/educacionMun.csv", index=False)